In [0]:
import dlt
from pyspark.sql.functions import *

#Bronze Load for MemberData CSV
@dlt.table(
    comment="Bronze Load for MemberData CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def BronzeMembers():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/members.csv")
    return df


#Bronze Load for Diagnosis CSV
@dlt.table(
    comment="Bronze Load for Diagnosis CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def BronzeDiagnosis():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/diagnosis_ref.csv")
    return df


#Bronze Load for Claim CSV
@dlt.table(
    comment="Bronze Load for Claim CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def BronzeClaim():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/claims_batch.csv")
    return df



The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4760633582222195>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import col
      4 @dlt.table(
      5   name="capstone_hospital.bronze_members",
      6   comment="Raw members data ingested to Bronze layer",
   (...)
     10 )
     11 def bronze_members():

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/autoreload_discoverability_hook.py:96, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     90 if not self._should_hint and (
     91     (module := sys.modules.get(absolute_name)) is not None and
     92     (fname := get_allowed_file_name_or_none(module)) is not None and
     93     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     94         absolute_name, float("inf")) and not self._should_hint):
     95     self._should_hint 

In [0]:
# Bronze Load for Claims Stream JSON
@dlt.table(
    comment="Bronze Load for Claims Stream JSON",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def BronzeClaimsStream():
    return (
        spark.read.format("json")
        .load("/Volumes/capstone_hospital/data/raw/claims_stream.json"))

# Bronze Load for Providers JSON
@dlt.table(
    comment="Bronze Load for Providers JSON",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def BronzeProviders():
    return (
        spark.read.format("json")
        .load("/Volumes/capstone_hospital/data/raw/providers.json")
    )



In [0]:
# SilverView: Member : cleaned data including quality checks
@dlt.table(
  name="SilverMembers",
  comment="Cleaned and deduplicated for Silver sales data",
   table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def SilverMembers():
    df = dlt.read("BronzeMembers")
    return df.filter(col("memberID").isNotNull()).dropDuplicates(["memberID"])

# SilverView: diagnosis : cleaned data including quality checks
@dlt.table(
    name="SilverDiagnosis",
    comment="Cleaned data including quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def SilverDiagnosis():
    return dlt.read("BronzeDiagnosis").filter(col("code").isNotNull()).dropDuplicates(["code"])

# SilverView: diagnosis : cleaned data including quality checks
@dlt.table(
    name="SilverClaim",
    comment="Cleaned data including quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def SilverClaim():
    return dlt.read("BronzeClaim").filter(col("ClaimID").isNotNull()).dropDuplicates(["ClaimID"])    

 # SilverView: claims stream : cleaned data including quality checks
@dlt.table(
    name="silveClaimsStream",
    comment="Cleaned data including quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def SilverClaim():
    return dlt.read("BronzeClaimsStream").filter(col("ClaimID").isNotNull()).dropDuplicates(["ClaimID"])    

 # SilverView: provider : cleaned data including quality checks
@dlt.table(
    name="SilverProviders",
    comment="Cleaned data including quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def SilverClaim():
    return dlt.read("BronzeProviders").filter(col("ProviderID").isNotNull()).dropDuplicates(["ProviderID"])    

In [0]:
@dlt.table(
    name="GoldFraudDetection",
    comment="Joined with Filtered data for Fraud Detection"
)
def GoldFraudDetection():
    # Load the tables
    SilverClaim = spark.read.table("capstone_hospital.default.SilverClaim")
    SilverDiagnosis = spark.read.table("capstone_hospital.default.SilverDiagnosis")
    SilverMembers = spark.read.table("capstone_hospital.default.SilverMembers")

    # Explode ICD10Codes into multiple rows
    exploded_claims = SilverClaim.withColumn("ICD10Code", explode(split(col("ICD10Codes"), ",")))

    # Filter rows where ClaimDate is more than 24 hours before ServiceDate
    filtered_claims = exploded_claims.filter(datediff(col("ClaimDate"), col("ServiceDate")) > 1)

    # Join the tables
    result = filtered_claims.join(SilverMembers, "MemberID") \
                            .join(silver_diagnosis, filtered_claims["ICD10Code"] == silver_diagnosis["code"])

    return result